In [45]:
SHP_FILE = '../../data/inputs/2308.gpkg'
DATA_DIR = "../../tmpdata"

In [46]:
import geopandas as gpd
from pathlib import Path
import os
import shutil

In [47]:
input_ = gpd.read_file(SHP_FILE)


In [48]:
TMP_DATA_DIR = f"{DATA_DIR}/dnq"
if os.path.exists(TMP_DATA_DIR):
    shutil.rmtree(TMP_DATA_DIR)
Path(TMP_DATA_DIR).mkdir(parents=True, exist_ok=True)

In [49]:
INTERIM_DIR = f'{TMP_DATA_DIR}/interim'
Path(INTERIM_DIR).mkdir(parents=True, exist_ok=True)
sh = ShapefileHelper(SHP_FILE, INTERIM_DIR)
sh.make_grid(50000, "grid");

--------- Generating grid at 50000mx50000m ...
--------- Grid generation complete. Saving to disk...
--------- Successfully saved to disk: ../../tmpdata/dnq/interim/grid.gpkg


,grid_id,geometry,DISTID,OBJECTID,PROV_34_NA,DIST_34_NA,PROVID,poppy_ha,ag_ha,poppy_perc,quantile,rank
0,0,"POLYGON ((63.33405 31.85963, 63.86031 31.88205...",2308,33,Hilmand,Nad Ali,23,15091,114729.18,0.1315358481599886,7,34
1,1,"POLYGON ((63.36129 31.41071, 63.88505 31.43275...",2308,33,Hilmand,Nad Ali,23,15091,114729.18,0.1315358481599886,7,34
2,2,"POLYGON ((63.86031 31.88205, 64.38712 31.90231...",2308,33,Hilmand,Nad Ali,23,15091,114729.18,0.1315358481599886,7,34
3,3,"POLYGON ((63.88505 31.43275, 64.40934 31.45266...",2308,33,Hilmand,Nad Ali,23,15091,114729.18,0.1315358481599886,7,34


In [50]:
GRID_PATH = f"{INTERIM_DIR}/grid.gpkg"
grid = gpd.read_file(GRID_PATH)

In [57]:
for i, tile in grid.iterrows():
    temp = gpd.GeoDataFrame(grid.iloc[i]).transpose()
    temp = temp[['grid_id', 'geometry']]
    temp['grid_id'] = temp['grid_id'].astype('int')
    temp.to_file(f'{INTERIM_DIR}/{temp.iloc[0][0]}.gpkg')

In [58]:
grids = [tile for tile in os.listdir(INTERIM_DIR) if tile != "grid.gpkg" and ]

In [59]:
grids

['.ipynb_checkpoints', '0.gpkg', '1.gpkg', '2.gpkg', '3.gpkg']

In [6]:
import geopandas as gpd
from osgeo import ogr
from math import ceil
import shutil
import os
from shapely.geometry import box
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")


# Shape file helper class
class ShapefileHelper():
    
    def __init__(self, dissolved_aoi_shapefile, vector_output_dir):
        self.raw = dissolved_aoi_shapefile
        self.vector_output_dir = vector_output_dir + "/"
        self._make_dir()
        self._read_file()
    
    def make_grid(self, resolution, name, out_crs="epsg:4326", id_col="grid_id"):
#         self.gdf.boundary.plot()
        
        gridWidth, gridHeight = resolution, resolution
        xmin, ymin, xmax, ymax = self.gdf.bounds.iloc[0]['minx'],  self.gdf.bounds.iloc[0]['miny'],  self.gdf.bounds.iloc[0]['maxx'],  self.gdf.bounds.iloc[0]['maxy']
        
        outputGridfn = name
        
        # get rows
        rows = ceil((ymax-ymin)/gridHeight)
        # get columns
        cols = ceil((xmax-xmin)/gridWidth)

        # start grid cell envelope
        ringXleftOrigin = xmin
        ringXrightOrigin = xmin + gridWidth
        ringYtopOrigin = ymax
        ringYbottomOrigin = ymax-gridHeight
        
        # create output file
        outDriver = ogr.GetDriverByName('ESRI Shapefile')
#         if os.path.exists(self.vector_output_dir + outputGridfn):
#             print("Deleting pre-existing shapefile: {}{}{}.shp".format(self.vector_output_dir,outputGridfn+"/", outputGridfn))
#             shutil.rmtree(self.vector_output_dir + outputGridfn,  ignore_errors=True)
            
#         if os.path.isdir(self.vector_output_dir):
#             print("Deleting pre-existing folder: {}{}/".format(self.vector_output_dir, outputGridfn))
#             shutil.rmtree(self.vector_output_dir + outputGridfn,  ignore_errors=True)
#         else:
#             os.makedirs(self.vector_output_dir)
#             print("Created new directory: {}{}/".format(self.vector_output_dir, outputGridfn))
            
            
        outDataSource = outDriver.CreateDataSource(self.vector_output_dir + outputGridfn + ".shp")
        outLayer = outDataSource.CreateLayer(outputGridfn,geom_type=ogr.wkbPolygon)
        featureDefn = outLayer.GetLayerDefn()
        
        print(f'--------- Generating grid at {resolution}mx{resolution}m ...')

        # create grid cells
        countcols = 0
        while countcols < cols:
            countcols += 1

            # reset envelope for rows
            ringYtop = ringYtopOrigin
            ringYbottom =ringYbottomOrigin
            countrows = 0

            while countrows < rows:
                countrows += 1
                ring = ogr.Geometry(ogr.wkbLinearRing)
                ring.AddPoint(ringXleftOrigin, ringYtop)
                ring.AddPoint(ringXrightOrigin, ringYtop)
                ring.AddPoint(ringXrightOrigin, ringYbottom)
                ring.AddPoint(ringXleftOrigin, ringYbottom)
                ring.AddPoint(ringXleftOrigin, ringYtop)
                poly = ogr.Geometry(ogr.wkbPolygon)
                poly.AddGeometry(ring)

                # add new geom to layer
                outFeature = ogr.Feature(featureDefn)
                outFeature.SetGeometry(poly)
                outLayer.CreateFeature(outFeature)
                outFeature.Destroy

                # new envelope for next poly
                ringYtop = ringYtop - gridHeight
                ringYbottom = ringYbottom - gridHeight

            # new envelope for next poly
            ringXleftOrigin = ringXleftOrigin + gridWidth
            ringXrightOrigin = ringXrightOrigin + gridWidth
        # Close DataSources
        outDataSource.Destroy()
        print('--------- Grid generation complete. Saving to disk...')        
        
        
#         # clip to boundary
        output = gpd.read_file(self.vector_output_dir + outputGridfn +".shp")
        
        output = output.set_crs("epsg:32642").to_crs(out_crs).reset_index().rename(columns={"index": id_col}).drop('FID', axis=1)
        output = gpd.sjoin(output, self.gdf.to_crs("epsg:4326")).drop('index_right', axis=1)
        
        
#         output = gpd.sjoin(output.to_crs("epsg:4326"), self.gdf).drop('index_right', axis=1)

        
#         print("Saving generated shapefile to disk..")
#         if os.path.exists(self.vector_output_dir + outputGridfn):
#             directory = self.vector_output_dir + outputGridfn
#             for f in os.listdir(directory):
#                 os.remove(os.path.join(directory, f))

        self.output = output
        output = output.drop(id_col, axis=1).reset_index().drop('index', axis=1).reset_index().rename(columns={'index': id_col})
        output.to_file(self.vector_output_dir + outputGridfn + ".gpkg", driver='GPKG')
        self._clean_dir()
        print("--------- Successfully saved to disk: {}".format(self.vector_output_dir + outputGridfn+ ".gpkg"))
        return output

    def subset_grid(self, grid_path, aoi_path):
        aoi = gpd.read_file(aoi_path)
        grid = gpd.read_file(grid_path)
        
        subset = gpd.sjoin(grid, aoi).drop('index_right', axis=1)
        subset.to_file(grid_path, driver='GPKG')
        
    def _read_file(self):
        raw = gpd.read_file(self.raw)
#         geom = box(*raw.total_bounds)
#         gdf = gpd.GeoDataFrame([{"geometry": geom}])
#         gdf = gdf.set_crs("epsg:4326")
        self.gdf = raw.to_crs("epsg:32642")
    
    def _make_dir(self):
        Path(self.vector_output_dir).mkdir(parents=True, exist_ok=True)
    
    def _clean_dir(self):
        test = os.listdir(self.vector_output_dir)

        for item in test:
            if item.endswith(".shp"):
                os.remove(os.path.join(self.vector_output_dir, item))
            if item.endswith(".shx"):
                os.remove(os.path.join(self.vector_output_dir, item))
            if item.endswith(".dbf"):
                os.remove(os.path.join(self.vector_output_dir, item))
        
